In [ ]:
#imports

import numpy as np
import xarray as xr
import scipy.sparse.linalg 




invm = np.linalg.inv
gmres = scipy.sparse.linalg.gmres
npsolve = np.linalg.solve

ds_1 = xr.open_dataset("data/regions_verify_isotope_202112_cov.nc")
bio_1 = ds_1["covariance_bio"]
anth_1 = ds_1["covariance_anth"]

ds_2 = xr.open_dataset("data/regions_verify_202104_cov.nc", chunks = 'auto')
bio_2 = ds_2["covariance_bio"]
anth_2 = ds_2["covariance_anth"]

M = bio_2.values


The "real" inverse
--------------------

In [ ]:
invM = invm(M)

np.linalg.solve
----------------
* ainakin näyttäisi tulevan samat sarakevektorit, mutta se ei (kai) käytä GMRESiä

In [ ]:
i = 4 #which column to solve
e_i = np.zeros(len(M))
e_i[i]= 1.

s_i = np.linalg.solve(M, e_i)
print(s_i)
c_i = invM[:,i]

print(np.allclose(c_i, s_i))

scipy.sparse.linalg.gmres
-----------------------------

* needs a smaller tolerance than the default for the resulting vector to be close to the actual column of the inverse
* A way to decrease the number of iterations is to use a preconditioner - maybe need to look into this if not fast enough!

In [ ]:
g_i, exitCode = gmres(M,e_i, tol = 1e-12)

print(g_i)
print(exitCode)

Comparison of times it takes to solve one column
--------------------------------------------------

* scipyn gmres a lot faster than numpy solve at least with tolerance 1e-12 or larger
* not as exact though (if we presume invm computes the correct inverse. (It could also be that invm uses the same np.linalg.solve and that is the reason it gives the same column as solve)

In [ ]:
i = 5 #which column to solve
e_i = np.zeros(len(M))
e_i[i]= 1.

%timeit npsolve(M, e_i)
%timeit gmres(M,e_i, tol = 1e-12)